In [ ]:
import requests
import pandas as pd
import importlib
import models
importlib.reload(models)
from models import Base, Tick, get_engine, get_sessionmaker
pd.set_option('display.expand_frame_repr', False) # display full data in terminal

engine = get_engine()
Session = get_sessionmaker(engine)

Base.metadata.create_all(engine)

In [25]:
# fetch data, save a raw csv. no need to run this cell if `ticks_raw.csv` alredy exists.
def fetch_mp_data(url) -> pd.DataFrame:
    '''
    MountainProject used to have an API to get tick data, but they shut it down. 
    But download urls are not authenticated so, just use download url you want.
    '''
    response = requests.get(tick_url)
    response.raise_for_status()
    with open('/app/csv_files/ticks_raw.csv', 'wb') as file:
        file.write(response.content)

tick_url = "https://www.mountainproject.com/user/107708526/aaron-glasenapp/tick-export"
fetch_mp_data(tick_url)

In [2]:
# use the dataframe to insert into Ticks table in postgres.
df_raw = pd.read_csv('/app/csv_files/ticks_raw.csv')
df = df_raw.copy()
df.columns = df.columns.str.replace(' ', '_').str.lower()

def insert_ticks_from_dataframe(df):
    session = Session()
    current_ticks = session.query(Tick).all()
    print(f'Currently {len(current_ticks)} ticks.')

    df.to_sql('ticks', con=engine, if_exists='append', index=False)

    current_ticks = session.query(Tick).all()
    print(f'After insert: {len(current_ticks)} ticks.')

    session.close()

insert_ticks_from_dataframe(df)

Currently 0 ticks.
After insert: 2560 ticks.
